<a href="https://colab.research.google.com/github/adriannaStaszewska/CS539Project/blob/main/CS539.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# uninstall improper package versions
!pip uninstall keras -y
!pip uninstall keras-nightly -y
!pip uninstall keras-Preprocessing -y
!pip uninstall keras-vis -y
!pip uninstall tensorflow -y
!pip uninstall h5py -y
!pip uninstall tensorflow-gpu -y

Found existing installation: keras 2.7.0
Uninstalling keras-2.7.0:
  Successfully uninstalled keras-2.7.0
Found existing installation: Keras-Preprocessing 1.1.2
Uninstalling Keras-Preprocessing-1.1.2:
  Successfully uninstalled Keras-Preprocessing-1.1.2
Found existing installation: keras-vis 0.4.1
Uninstalling keras-vis-0.4.1:
  Successfully uninstalled keras-vis-0.4.1
Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Successfully uninstalled tensorflow-2.7.0
Found existing installation: h5py 3.1.0
Uninstalling h5py-3.1.0:
  Successfully uninstalled h5py-3.1.0


In [ ]:
# reinstall with proper versions
!pip install tensorflow==1.13.1
!pip install tensorflow-gpu==1.13.1
!pip install keras==2.2.5
!pip install h5py==2.10.0
!pip install opencv-python
!pip install scikit-image==0.16.2
#!pip install mask-rcnn==2.1

     |████████████████████████████████| 92.6 MB 64.4 MB/s 
     |████████████████████████████████| 3.2 MB 18.6 MB/s 
     |████████████████████████████████| 367 kB 58.6 MB/s 
     |████████████████████████████████| 50 kB 9.4 MB/s 
     |████████████████████████████████| 42 kB 1.9 MB/s 
     |████████████████████████████████| 4.1 MB 68.0 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.6 requires tensorflow>=2.0.0, but you have tensorflow 1.13.1 which is incompatible.
     |

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available()

True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Mask_RCNN
!python setup.py install
!pip show mask-rcnn

/content/drive/MyDrive/Mask_RCNN
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'license-file' will not be supported in future versions. Please use the underscore name 'license_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'requirements-file' will not be supported in future versions. Please use the underscore name 'requirements_file' instead
  % (opt, underscore_opt))
running install
running bdist_egg
running egg_info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
reading manifest tem

In [ ]:
import json
import os
import sys
import numpy as np
import cv2
from mrcnn import utils
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
import skimage.draw
import urllib.request
from keras.callbacks import TensorBoard
import datetime
import tensorflow as tf

global CC 
CC = 0

######################################
# Configuration
######################################
class CustomConfig(Config):
    NAME = "custom_mcrnn"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 40
    STEPS_PER_EPOCH = 100
    VALIDATION_STEPS = 5
    LEARNING_RATE = .0025
    BATCH_SIZE = 10

config = CustomConfig()

#######################################
# Dataset
#######################################
class CustomDataset(utils.Dataset):
    # all annotation classes, to generate from the json
    CLASSES = ['i2','i4','i5','il100','il60','il80', 'io','ip','p10','p11','p12','p19','p23','p26','p27','p3','p5','pg','ph4',
           'ph4.5','pl100','pl120','pl20','pl30','pl40','pl5', 'pl50','pl60','pl70','pl80','pm20','pm55','pn','pne','po',
           'pr40','w13','w55','w57','w59'] 

    def load_dataset(self, subset):
        global CC
      
        #add all new classes
        for i, c in enumerate(self.CLASSES):
            self.add_class("category", i+1, self.CLASSES[i])

        DATASET_DIR = '/content/drive/MyDrive/data/'
        BASE_ANNOTATIONS_FILE = '/content/drive/MyDrive/data/annotations.json'
        annotations = json.load(open(BASE_ANNOTATIONS_FILE))
       
        #go through each image
        for i, a in enumerate(annotations["imgs"]):
            class_ids = []
            polygons = []

            # CC += 1
            # if CC > 100:
            #   print("breaking it")
            #   break

            image_path = os.path.join(DATASET_DIR, annotations['imgs'][a]['path'])
            trainOrTest = annotations['imgs'][a]['path'].split("/")[0]
            # print(annotations['imgs'][a]['path'])
            if trainOrTest == subset:
              if annotations['imgs'][a]['objects']:
              #if "polygon" in annotations['imgs'][a]['objects'].keys():
                for sign in annotations['imgs'][a]['objects']:
                  if sign['category'] in self.CLASSES: #not needed for augmented dataset
                    if "polygon" in sign:
                      polygons.append(["P", sign["polygon"]])
                      class_id = self.CLASSES.index(sign['category'])
                      class_ids.append(class_id)
                    elif "ellipse" in sign:
                      polygons.append(["E", sign["ellipse"]])
                      class_id = self.CLASSES.index(sign['category'])
                      class_ids.append(class_id)
      
                if os.path.exists(image_path):
                  print(image_path)
                  image = skimage.io.imread(image_path)
                  height, width = image.shape[:2]
                  self.add_image(
                            "category",
                            image_id=a,
                            path=image_path,
                            width=width, height=height,
                            polygons=polygons, class_ids=class_ids)


    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        info = self.image_info[image_id]
        class_ids = info['class_ids']

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        
        for i, p in enumerate(info["polygons"]):
            if p[0] == "P": #polygon mask
              all_x, all_y = zip(*p[1]) #unzip the list
              # Get indexes of pixels inside the polygon and set them to 1
              # rr, cc = skimage.draw.polygon(p[1], p[0])
              rr, cc = skimage.draw.polygon(all_y, all_x)
              rr[rr > mask.shape[0]-1] = mask.shape[0]-1
              cc[cc > mask.shape[1]-1] = mask.shape[1]-1
              mask[rr, cc, i] = 1
            else: #ellipse mask
              rr, cc = skimage.draw.ellipse(p[1][0][1], p[1][0][0], p[1][1][1], p[1][1][0], rotation=np.deg2rad(p[1][2]))
              rr[rr > mask.shape[0]-1] = mask.shape[0]-1
              cc[cc > mask.shape[1]-1] = mask.shape[1]-1
              mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance.
        return mask.astype(np.bool), np.array(class_ids, dtype=np.int32)


#######################################
# Training
#######################################
# print(os.path.abspath(os.getcwd()))
# define constants

TRAIN_TEST_SPLIT = .8 # proportion of images to use for training set, remainder will be reserved for validation

# set up train and validation data
dataset_train = CustomDataset()
dataset_train.load_dataset("train")
dataset_train.prepare()

dataset_val = CustomDataset()
dataset_val.load_dataset("test")
dataset_val.prepare()


#dataset_eval = CustomDataset() #we want to create this from 'other' folder 
#dataset_eval.load_dataset("eval")
#dataset_eval.prepare()


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

/content/drive/MyDrive/data/train/1967.jpg
/content/drive/MyDrive/data/train/10169.jpg
/content/drive/MyDrive/data/train/76132.jpg
/content/drive/MyDrive/data/train/10917.jpg
/content/drive/MyDrive/data/train/80772.jpg
/content/drive/MyDrive/data/train/49852.jpg
/content/drive/MyDrive/data/train/85040.jpg
/content/drive/MyDrive/data/train/22072.jpg
/content/drive/MyDrive/data/train/81523.jpg
/content/drive/MyDrive/data/train/92429.jpg
/content/drive/MyDrive/data/train/98489.jpg
/content/drive/MyDrive/data/train/45626.jpg
/content/drive/MyDrive/data/train/3397.jpg
/content/drive/MyDrive/data/train/39485.jpg
/content/drive/MyDrive/data/train/41063.jpg
/content/drive/MyDrive/data/train/36319.jpg
/content/drive/MyDrive/data/train/78384.jpg
/content/drive/MyDrive/data/train/46785.jpg
/content/drive/MyDrive/data/train/73557.jpg
/content/drive/MyDrive/data/train/81731.jpg
/content/drive/MyDrive/data/train/16766.jpg
/content/drive/MyDrive/data/train/94034.jpg
/content/drive/MyDrive/data/train/

In [ ]:

if not os.path.exists("mask_rcnn_coco.h5"):
    urllib.request.urlretrieve("https://github.com/matterport/Mask_RCNN/releases/download/v1.0/mask_rcnn_coco.h5", "mask_rcnn_coco.h5")

model = MaskRCNN(mode='training', model_dir='./model/', config=CustomConfig())
model.keras_model.metrics_tensors = []

# Tensorflow board
# logdir = os.path.join(
#     "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)
# print(tensorboard_callback)

model.load_weights("mask_rcnn_coco.h5", by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

# print summary
print(model.keras_model.summary())

# train model
model.train(train_dataset=dataset_train,
            val_dataset=dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=40,
            layers='heads')

# save training results to etenxternal file
model.keras_model.save_weights('model_40epochs_heads.h5')
model.keras_model.save_weights('/content/drive/MyDrive/model_40epochs_heads.h5')


Instructions for updating:
Colocations handled automatically by placer.
Model: "mask_rcnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_image[0][0]                
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNorm)            (None, None, None, 6 256         conv1[0][0]                      
__________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/40
100/100 [==============================] - 87s 866ms/step - loss: 2.3708 - rpn_class_loss: 0.0727 - rpn_bbox_loss: 0.6938 - mrcnn_class_loss: 0.2733 - mrcnn_bbox_loss: 0.6847 - mrcnn_mask_loss: 0.6463 - val_loss: 1.8909 - val_rpn_class_loss: 0.0829 - val_rpn_bbox_loss: 0.4578 - val_mrcnn_class_loss: 0.0423 - val_mrcnn_bbox_loss: 0.6166 - val_mrcnn_mask_loss: 0.6913
Epoch 2/40
100/100 [==============================] - 43s 431ms/step - loss: 1.9438 - rpn_class_loss: 0.0563 - rpn_bbox_loss: 0.4190 - mrcnn_class_loss: 0.2633 - mrcnn_bbox_loss: 0.5293 - mrcnn_mask_loss: 0.6759 - val_loss: 1.9831 - val_rpn_class_loss: 0.0429 - val_rpn_bbox_loss: 0.5011 - val_mrcnn_class_loss: 0.2291 - val_mrcnn_bbox_loss: 0.5838 - val_mrcnn_mask_loss: 0.6263
Epoch 3/40
100/100 [==============================] - 44s 444ms/step - loss: 1.8997 - rpn_class_loss: 0.0898 - rpn_bbox_loss: 0.5240 - mrcnn_class_loss: 0.2552 - mrcnn_bbox_loss: 0.4743 - mrcnn_mask_loss: 0.5565 - val_loss: 1.6448 - val_rpn_c

In [ ]:
model = MaskRCNN(mode='training', model_dir='./model/', config=CustomConfig())
model.keras_model.metrics_tensors = []
model.load_weights("model_40epochs_heads.h5", by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
model.train(train_dataset=dataset_train,
            val_dataset=dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=10,
            layers='all')
model.keras_model.save_weights('model_50epochs_all.h5')
model.keras_model.save_weights('/content/drive/MyDrive/model_50epochs.h5')

class CustomConfig2(Config):
    NAME = "custom_mcrnn"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 40
    STEPS_PER_EPOCH = 100
    VALIDATION_STEPS = 5
    LEARNING_RATE = .0005
    BATCH_SIZE = 10

config = CustomConfig2()

model = MaskRCNN(mode='training', model_dir='./model/', config=CustomConfig2())
model.keras_model.metrics_tensors = []
model.load_weights("model_50epochs_all.h5", by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
model.train(train_dataset=dataset_train,
            val_dataset=dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=30,
            layers='all')
model.keras_model.save_weights('model_80epochs.h5')
model.keras_model.save_weights('/content/drive/MyDrive/model_80epochs.h5')



Instructions for updating:
Colocations handled automatically by placer.

Starting at epoch 0. LR=0.0025

Checkpoint Path: ./model/custom_mcrnn20211214T1818/mask_rcnn_custom_mcrnn_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch

/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/10
100/100 [==============================] - 107s 1s/step - loss: 1.8181 - rpn_class_loss: 0.0293 - rpn_bbox_loss: 0.2200 - mrcnn_class_loss: 0.4190 - mrcnn_bbox_loss: 0.4907 - mrcnn_mask_loss: 0.6591 - val_loss: 2.9843 - val_rpn_class_loss: 0.0049 - val_rpn_bbox_loss: 0.1708 - val_mrcnn_class_loss: 0.8452 - val_mrcnn_bbox_loss: 0.7255 - val_mrcnn_mask_loss: 1.2378
Epoch 2/10
100/100 [==============================] - 57s 567ms/step - loss: 1.5287 - rpn_class_loss: 0.0196 - rpn_bbox_loss: 0.2567 - mrcnn_class_loss: 0.3620 - mrcnn_bbox_loss: 0.3825 - mrcnn_mask_loss: 0.5078 - val_loss: 1.7674 - val_rpn_class_loss: 0.0106 - val_rpn_bbox_loss: 0.0867 - val_mrcnn_class_loss: 0.2159 - val_mrcnn_bbox_loss: 0.4549 - val_mrcnn_mask_loss: 0.9993
Epoch 3/10
100/100 [==============================] - 57s 566ms/step - loss: 1.2752 - rpn_class_loss: 0.0118 - rpn_bbox_loss: 0.2081 - mrcnn_class_loss: 0.3153 - mrcnn_bbox_loss: 0.3315 - mrcnn_mask_loss: 0.4085 - val_loss: 1.4386 - val_rpn_cla

In [ ]:
class CustomConfig2(Config):
    NAME = "custom_mcrnn"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 40
    STEPS_PER_EPOCH = 100
    VALIDATION_STEPS = 5
    LEARNING_RATE = .001
    BATCH_SIZE = 5

config = CustomConfig2()

model = MaskRCNN(mode='training', model_dir='./model/', config=CustomConfig2())
model.keras_model.metrics_tensors = []
model.load_weights("model_80epochs.h5", by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
model.train(train_dataset=dataset_train,
            val_dataset=dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=40,
            layers='heads')
model.keras_model.save_weights('model_120epochs.h5')
model.keras_model.save_weights('/content/drive/MyDrive/model_120epochs.h5')



Starting at epoch 0. LR=0.001

Checkpoint Path: ./model/custom_mcrnn20211214T1905/mask_rcnn_custom_mcrnn_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4   

/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/40
100/100 [==============================] - 75s 750ms/step - loss: 1.9172 - rpn_class_loss: 0.0152 - rpn_bbox_loss: 0.2321 - mrcnn_class_loss: 0.4799 - mrcnn_bbox_loss: 0.5276 - mrcnn_mask_loss: 0.6624 - val_loss: 2.2584 - val_rpn_class_loss: 0.0155 - val_rpn_bbox_loss: 0.3317 - val_mrcnn_class_loss: 0.3944 - val_mrcnn_bbox_loss: 0.5477 - val_mrcnn_mask_loss: 0.9692
Epoch 2/40
100/100 [==============================] - 37s 369ms/step - loss: 1.6490 - rpn_class_loss: 0.0120 - rpn_bbox_loss: 0.1942 - mrcnn_class_loss: 0.4014 - mrcnn_bbox_loss: 0.4708 - mrcnn_mask_loss: 0.5706 - val_loss: 2.6487 - val_rpn_class_loss: 0.0183 - val_rpn_bbox_loss: 0.1619 - val_mrcnn_class_loss: 0.9865 - val_mrcnn_bbox_loss: 0.5499 - val_mrcnn_mask_loss: 0.9322
Epoch 3/40
100/100 [==============================] - 36s 362ms/step - loss: 1.3490 - rpn_class_loss: 0.0122 - rpn_bbox_loss: 0.2517 - mrcnn_class_loss: 0.3849 - mrcnn_bbox_loss: 0.3881 - mrcnn_mask_loss: 0.3121 - val_loss: 1.8931 - val_rpn_c

In [ ]:
class CustomConfig2(Config):
    NAME = "custom_mcrnn"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 40
    STEPS_PER_EPOCH = 100
    VALIDATION_STEPS = 5
    LEARNING_RATE = .001
    BATCH_SIZE = 5

config = CustomConfig2()

model = MaskRCNN(mode='training', model_dir='./model/', config=CustomConfig2())
model.keras_model.metrics_tensors = []
model.load_weights("model_120epochs.h5", by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
model.train(train_dataset=dataset_train,
            val_dataset=dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=40,
            layers='heads')
model.keras_model.save_weights('model_160epochs.h5')
model.keras_model.save_weights('/content/drive/MyDrive/model_160epochs.h5')


Starting at epoch 0. LR=0.001

Checkpoint Path: ./model/custom_mcrnn20211214T1939/mask_rcnn_custom_mcrnn_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4   

/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/40
100/100 [==============================] - 77s 767ms/step - loss: 2.1867 - rpn_class_loss: 0.0156 - rpn_bbox_loss: 0.2189 - mrcnn_class_loss: 0.5832 - mrcnn_bbox_loss: 0.5084 - mrcnn_mask_loss: 0.8606 - val_loss: 1.4693 - val_rpn_class_loss: 0.0050 - val_rpn_bbox_loss: 0.0568 - val_mrcnn_class_loss: 0.3363 - val_mrcnn_bbox_loss: 0.5586 - val_mrcnn_mask_loss: 0.5127
Epoch 2/40
100/100 [==============================] - 37s 370ms/step - loss: 1.6183 - rpn_class_loss: 0.0382 - rpn_bbox_loss: 0.3353 - mrcnn_class_loss: 0.2151 - mrcnn_bbox_loss: 0.5172 - mrcnn_mask_loss: 0.5125 - val_loss: 1.6156 - val_rpn_class_loss: 0.0029 - val_rpn_bbox_loss: 0.1048 - val_mrcnn_class_loss: 0.5041 - val_mrcnn_bbox_loss: 0.6124 - val_mrcnn_mask_loss: 0.3914
Epoch 3/40
100/100 [==============================] - 37s 373ms/step - loss: 1.2585 - rpn_class_loss: 0.0107 - rpn_bbox_loss: 0.1813 - mrcnn_class_loss: 0.2330 - mrcnn_bbox_loss: 0.3988 - mrcnn_mask_loss: 0.4346 - val_loss: 1.0504 - val_rpn_c

In [ ]:
import time
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

class InferenceConfig(Config):
    NAME = 'inference'
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 40
    DETECTION_MIN_CONFIDENCE = 0.5

inference_config = InferenceConfig()

CLASS_NAMES = ['i2','i4','i5','il100','il60','il80', 'io','ip','p10','p11','p12','p19','p23','p26','p27','p3','p5','pg','ph4',
           'ph4.5','pl100','pl120','pl20','pl30','pl40','pl5', 'pl50','pl60','pl70','pl80','pm20','pm55','pn','pne','po',
           'pr40','w13','w55','w57','w59'] 

model = MaskRCNN(mode='inference',
                 config=InferenceConfig(), model_dir='./')

model.load_weights(filepath='/content/drive/MyDrive/model_160epochs.h5', by_name=True)

image_ids = np.random.choice(dataset_val.image_ids, 50)
print(image_ids)
APs = []
times = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    time0 = time.time()
    results = model.detect([image], verbose=0)
    time1 = time.time()
    times.append(time1-time0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    #visualize.plot_precision_recall(AP, precisions, recalls)
    #visualize.display_instances(image=image,
                                 # boxes=r['rois'],
                                  #masks=r['masks'],
                                 # class_ids=r['class_ids'],
                                 # class_names=CLASS_NAMES,
                                  #scores=r['scores'])

    APs.append(AP)
    
print("mAP: ", np.mean(APs))
print("mean time: ", np.mean(times))

[1277 1237 1219 1283  794  443  274  928  612 1066 1191  506 1393 1204
 1283  767  860  290  495 1123  421 1347  267   42    6  334  552  568
  293  312   21 1252  548  958  424  368  712  995  903  550 1285 1286
 1046  733  113  510  628  684 1283 1373]
mAP:  0.045111111402511596
mean time:  0.23287664413452147
